In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
# Import libraries

import pandas as pd
import numpy as np
import random
import pyomo.environ as pe
from sklearn.preprocessing import MinMaxScaler
from MCDAfunctions import bwm_opt_fn, Step


### Get the weights for the micro proftabiility criteria

In [34]:
############################################################################
n = 3
best = 0
worst = 2
b_comparisons = np.array([1,3,6])
w_comparisons = np.array([6,2,1])

############################################################################
M = bwm_opt_fn(n, best, worst, b_comparisons, w_comparisons)
pe.SolverFactory('cplex').solve(M)
var_list = [pe.value(v) for v in  M.component_data_objects([pe.Var])]
weights_prof_micro = np.array(var_list[:-1]) 
weights_prof_micro = np.array([weights_prof_micro[2], weights_prof_micro[1], weights_prof_micro[0]])
weights_prof_micro

array([0.11111111, 0.22222222, 0.66666667])

### Get the initial (original) results

In [35]:
data = pd.read_excel("data.xlsx", index_col=0)
b_comparisons = np.array([1,9,4,5,1])
w_comparisons = np.array([9,1,6,5,9])
original_results = Step(b_comparisons, w_comparisons, data, weights_prof_micro)
original_results

,Value,Position
Alternative,,
a1,0.000000,42.0
a2,0.617706,17.0
a3,0.303760,35.0
a4,0.278327,36.0
a5,0.506785,25.0
a6,0.192340,39.0
a7,0.832935,4.0
a8,0.781046,6.0
a9,0.227917,38.0


### Post-Analysis Simulations

In [36]:
data = pd.read_excel("data.xlsx", index_col=0)
random.seed(10)
sim_results = [] 
n_sim=1000
for sim in range (n_sim):
    b_comparisons = np.array([1, random.choice([9,8]), random.choice([3,4,5]), random.choice([4,5,6]), 1])
    w_comparisons = np.array([random.choice([9,8]),1,random.choice([5,6,7]),random.choice([4,5,6]),random.choice([9,8])])
    table = Step(b_comparisons, w_comparisons, data, weights_prof_micro)   
    sim_results.append(table)     # Append to the simulation list of results


In [37]:
values_sim = pd.concat([sim_results[s].Value for s in range(n_sim)], axis = 1)
positions_sim = pd.concat([sim_results[s].Position for s in range(n_sim)], axis = 1)

Post_analysis = pd.concat([values_sim.mean(axis = 1), 
                            values_sim.std(axis = 1),
                            positions_sim.mode(axis = 1), 
                            positions_sim.min(axis=1),
                            positions_sim.max(axis=1)], axis=1)

Post_analysis.columns = ["Mean Global Value", "Standard Deviation", "Position Mode", "Best Position", "Worst Position"]

In [38]:
Result = pd.concat([table, Post_analysis], axis = 1)
Result

,Value,Position,Mean Global Value,Standard Deviation,Position Mode,Best Position,Worst Position
Alternative,,,,,,,
a1,0.000000,42.0,0.008290,1.626146e-02,42.0,40.0,42.0
a2,0.616033,20.0,0.635259,2.966560e-02,16.0,14.0,20.0
a3,0.301014,35.0,0.321143,2.320447e-02,35.0,34.0,36.0
a4,0.293384,36.0,0.294662,2.712575e-02,36.0,35.0,37.0
a5,0.513029,25.0,0.519872,2.232796e-02,25.0,24.0,28.0
a6,0.231306,39.0,0.192789,4.472403e-02,39.0,37.0,39.0
a7,0.832232,4.0,0.861268,4.424219e-02,4.0,3.0,5.0
a8,0.785237,6.0,0.805591,3.945205e-02,6.0,6.0,8.0
a9,0.251550,38.0,0.240957,2.795572e-02,38.0,38.0,39.0


In [40]:
#Result.to_excel("Results.xlsx")